In [1]:
import ollama # For running LLMs locally
from langchain_community.document_loaders import TextLoader # For loading text documents for use in LLMs
from langchain_community.document_loaders import DirectoryLoader # For loading directories of text documents for use in LLMs
import magic # This is needed for some reason for the DirectoryLoader to work
from ollama import generate
import pandas as pd
import json

In [2]:
# Do that weird "r" thing because paths are weird in Windows
path = r"C:\Users\MCOB PHD 14\Dropbox\Charlie's Dissertation\Beige Books\test_texts"

# Give it the path and tell it to look for .txt files
loader = DirectoryLoader(path, glob="**/*.txt")

# Read in all the documents from that directory as a list
docs = loader.load()

# Phi3

In [8]:
# Import necessary libraries
import json
import pandas as pd

# Set the local model and new prompt for classification
local_model = 'phi3'
prompt_sentiment = 'Label the overall economic sentiment in the following Beige Book text as negative, mixed, or positive. Return the label as a JSON value with a key of "label" and nothing else: '

# Function to get sentiment label for each document
def get_sentiment_label(doc_content):
    # Combine the sentiment prompt with the document content
    full_prompt = f"{prompt_sentiment}\n{doc_content}"
    # Generate the sentiment label using the local model
    result = ollama.generate(model=local_model, prompt=full_prompt, format='json', stream=False, options={'temperature': 0})
    return result

# Possible keys that might hold the sentiment label
label_keys = ["label", "sentiment_label"]

# List to store the data for the DataFrame
data = []

# Total number of documents for progress tracking
total_docs = len(docs)

# Collect sentiment results for each document, with document index and status update
for i, doc in enumerate(docs):
    doc_content = doc.page_content
    result = get_sentiment_label(doc_content)
    
    # Extract the response value from the result
    response_json = result.get('response', '{}')
    # Parse the response JSON if it's a string
    response_data = json.loads(response_json) if isinstance(response_json, str) else response_json
    
    # Extract the sentiment label using the possible keys
    sentiment_label = None
    for key in label_keys:
        if key in response_data:
            sentiment_label = response_data[key]
            break
    
    # If no label was found, set it as 'N/A'
    if sentiment_label is None:
        sentiment_label = 'N/A'
    
    # Append the result to the data list with the document index and content
    data.append({
        "Document Number": i + 1,
        "Document Content": doc_content,
        "Sentiment Label": sentiment_label,
        "Full Result": result
    })

    # Print status update
    print(f"Processed document {i + 1}/{total_docs} - Sentiment Label: {sentiment_label}")

# Create a DataFrame from the data list
df_phi3 = pd.DataFrame(data)

# Print the DataFrame
print(df_phi3)

Processed document 1/200 - Sentiment Label: negative
Processed document 2/200 - Sentiment Label: mixed
Processed document 3/200 - Sentiment Label: mixed
Processed document 4/200 - Sentiment Label: mixed
Processed document 5/200 - Sentiment Label: mixed
Processed document 6/200 - Sentiment Label: positive
Processed document 7/200 - Sentiment Label: positive
Processed document 8/200 - Sentiment Label: mixed
Processed document 9/200 - Sentiment Label: mixed
Processed document 10/200 - Sentiment Label: negative
Processed document 11/200 - Sentiment Label: negative
Processed document 12/200 - Sentiment Label: mixed
Processed document 13/200 - Sentiment Label: positive
Processed document 14/200 - Sentiment Label: mixed
Processed document 15/200 - Sentiment Label: negative
Processed document 16/200 - Sentiment Label: mixed
Processed document 17/200 - Sentiment Label: mixed
Processed document 18/200 - Sentiment Label: mixed
Processed document 19/200 - Sentiment Label: mixed
Processed document 

In [9]:
# Read in file names in the folder C:\Users\MCOB PHD 14\Dropbox\Charlie's Dissertation\Beige Books\test_texts and save as a column in df_phi3 named "file_names""1970_ch (1)_chunk_4_sentiment_scores.csv"
import os
import pandas as pd

# Correctly formatted directory path
directory = r"C:\Users\MCOB PHD 14\Dropbox\Charlie's Dissertation\Beige Books\test_texts"

# List all txt files in the directory
file_list = [f for f in os.listdir(directory) if f.endswith('.txt')]

# Create a dataframe with the list of file names
df_phi3['file_names'] = file_list

# Display the dataframe
df_phi3.head()


,Document Number,Document Content,Sentiment Label,Full Result,file_names
0,1,rates and terms have not eased much . There ar...,negative,"{'model': 'phi3', 'created_at': '2024-09-21T17...",1970_ch (1)_chunk_4.txt
1,2,"July 15 , 1970 Economic activity in the Fourth...",mixed,"{'model': 'phi3', 'created_at': '2024-09-21T17...",1970_cl (6)_chunk_1.txt
2,3,wage settlements and over continued upward pre...,mixed,"{'model': 'phi3', 'created_at': '2024-09-21T17...",1970_ny (1)_chunk_3.txt
3,4,adversely affected by curtailment of Governmen...,mixed,"{'model': 'phi3', 'created_at': '2024-09-21T17...",1971_kc (12)_chunk_2.txt
4,5,build offshore nuclear generators . Two wareho...,mixed,"{'model': 'phi3', 'created_at': '2024-09-21T17...",1972_at (9)_chunk_2.txt


In [11]:
# Load the sentiment scores CSV
excel_path = r"C:/Users/MCOB PHD 14/Dropbox/Charlie's Dissertation/Beige Books/manual_sentiment.csv"
human_df = pd.read_csv(excel_path)

# Define the label function
def label_sentiment(score):
    if score <= -0.3:
        return 0  # Negative
    elif score <= 0.2:
        return 1  # Mixed
    else:
        return 2  # Positive

# Apply the label function to the sentiment scores
human_df['label'] = human_df['human_sentiment'].apply(label_sentiment)

# convert label numbers to be negative, mixed, or positive
human_df['Human_Label'] = human_df['label'].replace({0: 'negative', 1: 'mixed', 2: 'positive'})

human_df.head()

# Merge the human sentiment labels with df_phi3 on file_names
merged_df = df_phi3.merge(human_df, how='inner', left_on='file_names', right_on='file_names')
merged_df.head()

,Document Number,Document Content,Sentiment Label,Full Result,file_names,Document,human_sentiment,scorer,label,Human_Label
0,1,rates and terms have not eased much . There ar...,negative,"{'model': 'phi3', 'created_at': '2024-09-21T17...",1970_ch (1)_chunk_4.txt,3,-0.5,CS,0,negative
1,2,"July 15 , 1970 Economic activity in the Fourth...",mixed,"{'model': 'phi3', 'created_at': '2024-09-21T17...",1970_cl (6)_chunk_1.txt,6,0.3,CS,2,positive
2,3,wage settlements and over continued upward pre...,mixed,"{'model': 'phi3', 'created_at': '2024-09-21T17...",1970_ny (1)_chunk_3.txt,13,-0.2,CS,1,mixed
3,4,adversely affected by curtailment of Governmen...,mixed,"{'model': 'phi3', 'created_at': '2024-09-21T17...",1971_kc (12)_chunk_2.txt,21,0.1,CS,1,mixed
4,5,build offshore nuclear generators . Two wareho...,mixed,"{'model': 'phi3', 'created_at': '2024-09-21T17...",1972_at (9)_chunk_2.txt,35,0.6,CS,2,positive


In [12]:
from sklearn.metrics import classification_report, accuracy_score

# Generate classification report
class_report = classification_report(merged_df['Human_Label'], merged_df['Sentiment Label'], labels=['positive', 'negative', 'mixed'])
print("\nClassification Report:")
print(class_report)

# Calculate and print accuracy
accuracy = accuracy_score(merged_df['Human_Label'], merged_df['Sentiment Label'])
print(f'Accuracy: {accuracy:.3f}')


Classification Report:
              precision    recall  f1-score   support

    positive       0.93      0.36      0.52        78
    negative       0.83      0.49      0.61        39
       mixed       0.52      0.93      0.67        83

    accuracy                           0.62       200
   macro avg       0.76      0.59      0.60       200
weighted avg       0.74      0.62      0.60       200

Accuracy: 0.620


# Gemma 2

In [13]:
# Import necessary libraries
import json
import pandas as pd

# Set the local model and new prompt for classification
local_model = 'gemma2'
prompt_sentiment = 'Label the overall economic sentiment in the following Beige Book text as negative, mixed, or positive. Return the label as a JSON value with a key of "label" and nothing else: '

# Function to get sentiment label for each document
def get_sentiment_label(doc_content):
    # Combine the sentiment prompt with the document content
    full_prompt = f"{prompt_sentiment}\n{doc_content}"
    # Generate the sentiment label using the local model
    result = ollama.generate(model=local_model, prompt=full_prompt, format='json', stream=False, options={'temperature': 0})
    return result

# Possible keys that might hold the sentiment label
label_keys = ["label", "sentiment_label"]

# List to store the data for the DataFrame
data = []

# Total number of documents for progress tracking
total_docs = len(docs)

# Collect sentiment results for each document, with document index and status update
for i, doc in enumerate(docs):
    doc_content = doc.page_content
    result = get_sentiment_label(doc_content)
    
    # Extract the response value from the result
    response_json = result.get('response', '{}')
    # Parse the response JSON if it's a string
    response_data = json.loads(response_json) if isinstance(response_json, str) else response_json
    
    # Extract the sentiment label using the possible keys
    sentiment_label = None
    for key in label_keys:
        if key in response_data:
            sentiment_label = response_data[key]
            break
    
    # If no label was found, set it as 'N/A'
    if sentiment_label is None:
        sentiment_label = 'N/A'
    
    # Append the result to the data list with the document index and content
    data.append({
        "Document Number": i + 1,
        "Document Content": doc_content,
        "Sentiment Label": sentiment_label,
        "Full Result": result
    })

    # Print status update
    print(f"Processed document {i + 1}/{total_docs} - Sentiment Label: {sentiment_label}")

# Create a DataFrame from the data list
df_gemma2 = pd.DataFrame(data)

# Print the DataFrame
df_gemma2.head()

Processed document 1/200 - Sentiment Label: negative
Processed document 2/200 - Sentiment Label: mixed
Processed document 3/200 - Sentiment Label: mixed
Processed document 4/200 - Sentiment Label: mixed
Processed document 5/200 - Sentiment Label: mixed
Processed document 6/200 - Sentiment Label: positive
Processed document 7/200 - Sentiment Label: positive
Processed document 8/200 - Sentiment Label: mixed
Processed document 9/200 - Sentiment Label: mixed
Processed document 10/200 - Sentiment Label: mixed
Processed document 11/200 - Sentiment Label: negative
Processed document 12/200 - Sentiment Label: mixed
Processed document 13/200 - Sentiment Label: positive
Processed document 14/200 - Sentiment Label: mixed
Processed document 15/200 - Sentiment Label: negative
Processed document 16/200 - Sentiment Label: mixed
Processed document 17/200 - Sentiment Label: mixed
Processed document 18/200 - Sentiment Label: mixed
Processed document 19/200 - Sentiment Label: mixed
Processed document 20/

,Document Number,Document Content,Sentiment Label,Full Result
0,1,rates and terms have not eased much . There ar...,negative,"{'model': 'gemma2', 'created_at': '2024-09-21T..."
1,2,"July 15 , 1970 Economic activity in the Fourth...",mixed,"{'model': 'gemma2', 'created_at': '2024-09-21T..."
2,3,wage settlements and over continued upward pre...,mixed,"{'model': 'gemma2', 'created_at': '2024-09-21T..."
3,4,adversely affected by curtailment of Governmen...,mixed,"{'model': 'gemma2', 'created_at': '2024-09-21T..."
4,5,build offshore nuclear generators . Two wareho...,mixed,"{'model': 'gemma2', 'created_at': '2024-09-21T..."


In [30]:
import os
import pandas as pd

# Create a dataframe with the list of file names
df_gemma2['file_names'] = file_list

# Display the dataframe
df_gemma2.head()

# Rename Sentiment Label as Gemma2_sentiment
df_gemma2.rename(columns={'Sentiment Label': 'Gemma2_sentiment'}, inplace=True)

# Merge the human sentiment labels with df_gemma2 on file_names
merged_df = human_df.merge(df_gemma2, how='inner', on='file_names')

In [36]:
# Generate classification report
class_report = classification_report(merged_df['Human_Label'], merged_df['Gemma2_sentiment'], labels=['positive', 'negative', 'mixed'])
print("\nClassification Report:")
print(class_report)

# Calculate and print accuracy
accuracy = accuracy_score(merged_df['Human_Label'], merged_df['Gemma2_sentiment'])
print(f'Accuracy: {accuracy:.3f}')


Classification Report:
              precision    recall  f1-score   support

    positive       0.96      0.29      0.45        78
    negative       0.84      0.41      0.55        39
       mixed       0.50      0.95      0.66        83

    accuracy                           0.59       200
   macro avg       0.77      0.55      0.55       200
weighted avg       0.75      0.59      0.56       200

Accuracy: 0.590


# Llama 3.1

In [14]:
# Import necessary libraries
import json
import pandas as pd

# Set the local model and new prompt for classification
local_model = 'llama3.1:latest'
prompt_sentiment = 'Label the overall economic sentiment in the following Beige Book text as negative, mixed, or positive. Return the label as a JSON value with a key of "label" and nothing else: '

# Function to get sentiment label for each document
def get_sentiment_label(doc_content):
    # Combine the sentiment prompt with the document content
    full_prompt = f"{prompt_sentiment}\n{doc_content}"
    # Generate the sentiment label using the local model
    result = ollama.generate(model=local_model, prompt=full_prompt, format='json', stream=False, options={'temperature': 0})
    return result

# Possible keys that might hold the sentiment label
label_keys = ["label", "sentiment_label"]

# List to store the data for the DataFrame
data = []

# Total number of documents for progress tracking
total_docs = len(docs)

# Collect sentiment results for each document, with document index and status update
for i, doc in enumerate(docs):
    doc_content = doc.page_content
    result = get_sentiment_label(doc_content)
    
    # Extract the response value from the result
    response_json = result.get('response', '{}')
    # Parse the response JSON if it's a string
    response_data = json.loads(response_json) if isinstance(response_json, str) else response_json
    
    # Extract the sentiment label using the possible keys
    sentiment_label = None
    for key in label_keys:
        if key in response_data:
            sentiment_label = response_data[key]
            break
    
    # If no label was found, set it as 'N/A'
    if sentiment_label is None:
        sentiment_label = 'N/A'
    
    # Append the result to the data list with the document index and content
    data.append({
        "Document Number": i + 1,
        "Document Content": doc_content,
        "Sentiment Label": sentiment_label,
        "Full Result": result
    })

    # Print status update
    print(f"Processed document {i + 1}/{total_docs} - Sentiment Label: {sentiment_label}")

# Create a DataFrame from the data list
df_llama = pd.DataFrame(data)

# Print the DataFrame
print(df_llama)

Processed document 1/200 - Sentiment Label: negative
Processed document 2/200 - Sentiment Label: mixed
Processed document 3/200 - Sentiment Label: mixed
Processed document 4/200 - Sentiment Label: mixed
Processed document 5/200 - Sentiment Label: mixed
Processed document 6/200 - Sentiment Label: positive
Processed document 7/200 - Sentiment Label: positive
Processed document 8/200 - Sentiment Label: mixed
Processed document 9/200 - Sentiment Label: mixed
Processed document 10/200 - Sentiment Label: negative
Processed document 11/200 - Sentiment Label: negative
Processed document 12/200 - Sentiment Label: negative
Processed document 13/200 - Sentiment Label: positive
Processed document 14/200 - Sentiment Label: mixed
Processed document 15/200 - Sentiment Label: negative
Processed document 16/200 - Sentiment Label: mixed
Processed document 17/200 - Sentiment Label: mixed
Processed document 18/200 - Sentiment Label: positive
Processed document 19/200 - Sentiment Label: negative
Processed 

In [39]:
df_llama['file_names'] = file_list

# Rename Sentiment Label as Llama_sentiment
df_llama.rename(columns={'Sentiment Label': 'Llama_sentiment'}, inplace=True)

# Merge the human sentiment labels with df_llama on file_names
merged_df = human_df.merge(df_llama, how='inner', on='file_names')

# Generate classification report
class_report = classification_report(merged_df['Human_Label'], merged_df['Llama_sentiment'], labels=['positive', 'negative', 'mixed'])
print("\nClassification Report:")
print(class_report)

# Calculate and print accuracy
accuracy = accuracy_score(merged_df['Human_Label'], merged_df['Llama_sentiment'])
print(f'Accuracy: {accuracy:.3f}')


Classification Report:
              precision    recall  f1-score   support

    positive       0.88      0.47      0.62        78
    negative       0.85      0.59      0.70        39
       mixed       0.57      0.90      0.70        83

    accuracy                           0.68       200
   macro avg       0.77      0.66      0.67       200
weighted avg       0.75      0.68      0.67       200

Accuracy: 0.675


In [40]:
merged_df.head()

,Document,file_names,human_sentiment,scorer,label,Human_Label,Document Number,Document Content,Llama_sentiment,Full Result
0,3,1970_ch (1)_chunk_4.txt,-0.5,CS,0,negative,1,rates and terms have not eased much . There ar...,negative,"{'model': 'llama3.1:latest', 'created_at': '20..."
1,6,1970_cl (6)_chunk_1.txt,0.3,CS,2,positive,2,"July 15 , 1970 Economic activity in the Fourth...",mixed,"{'model': 'llama3.1:latest', 'created_at': '20..."
2,13,1970_ny (1)_chunk_3.txt,-0.2,CS,1,mixed,3,wage settlements and over continued upward pre...,mixed,"{'model': 'llama3.1:latest', 'created_at': '20..."
3,21,1971_kc (12)_chunk_2.txt,0.1,CS,1,mixed,4,adversely affected by curtailment of Governmen...,mixed,"{'model': 'llama3.1:latest', 'created_at': '20..."
4,35,1972_at (9)_chunk_2.txt,0.6,CS,2,positive,5,build offshore nuclear generators . Two wareho...,mixed,"{'model': 'llama3.1:latest', 'created_at': '20..."


In [42]:
merged_df['Sentiment_Phi3'] = df_phi3['Sentiment Label']
merged_df['Sentiment_Gemma2'] = df_gemma2['Gemma2_sentiment']

# Export merged_df to a CSV file
#merged_df.to_csv('open_source_LLMs_classification.csv', index=False)

,Document,file_names,human_sentiment,scorer,label,Human_Label,Document Number,Document Content,Llama_sentiment,Full Result,Sentiment_Phi3
0,3,1970_ch (1)_chunk_4.txt,-0.5,CS,0,negative,1,rates and terms have not eased much . There ar...,negative,"{'model': 'llama3.1:latest', 'created_at': '20...",negative
1,6,1970_cl (6)_chunk_1.txt,0.3,CS,2,positive,2,"July 15 , 1970 Economic activity in the Fourth...",mixed,"{'model': 'llama3.1:latest', 'created_at': '20...",mixed
2,13,1970_ny (1)_chunk_3.txt,-0.2,CS,1,mixed,3,wage settlements and over continued upward pre...,mixed,"{'model': 'llama3.1:latest', 'created_at': '20...",mixed
3,21,1971_kc (12)_chunk_2.txt,0.1,CS,1,mixed,4,adversely affected by curtailment of Governmen...,mixed,"{'model': 'llama3.1:latest', 'created_at': '20...",mixed
4,35,1972_at (9)_chunk_2.txt,0.6,CS,2,positive,5,build offshore nuclear generators . Two wareho...,mixed,"{'model': 'llama3.1:latest', 'created_at': '20...",mixed


In [45]:
merged_df.head()


,Document,file_names,human_sentiment,scorer,label,Human_Label,Document Number,Document Content,Llama_sentiment,Full Result,Sentiment_Phi3,Sentiment_Gemma2
0,3,1970_ch (1)_chunk_4.txt,-0.5,CS,0,negative,1,rates and terms have not eased much . There ar...,negative,"{'model': 'llama3.1:latest', 'created_at': '20...",negative,negative
1,6,1970_cl (6)_chunk_1.txt,0.3,CS,2,positive,2,"July 15 , 1970 Economic activity in the Fourth...",mixed,"{'model': 'llama3.1:latest', 'created_at': '20...",mixed,mixed
2,13,1970_ny (1)_chunk_3.txt,-0.2,CS,1,mixed,3,wage settlements and over continued upward pre...,mixed,"{'model': 'llama3.1:latest', 'created_at': '20...",mixed,mixed
3,21,1971_kc (12)_chunk_2.txt,0.1,CS,1,mixed,4,adversely affected by curtailment of Governmen...,mixed,"{'model': 'llama3.1:latest', 'created_at': '20...",mixed,mixed
4,35,1972_at (9)_chunk_2.txt,0.6,CS,2,positive,5,build offshore nuclear generators . Two wareho...,mixed,"{'model': 'llama3.1:latest', 'created_at': '20...",mixed,mixed


In [46]:
# Do separate class reports for each model
class_report_phi3 = classification_report(merged_df['Human_Label'], merged_df['Sentiment_Phi3'], labels=['positive', 'negative', 'mixed'])
class_report_gemma2 = classification_report(merged_df['Human_Label'], merged_df['Sentiment_Gemma2'], labels=['positive', 'negative', 'mixed'])
class_report_llama = classification_report(merged_df['Human_Label'], merged_df['Llama_sentiment'], labels=['positive', 'negative', 'mixed'])

# Print the classification reports
print("\nClassification Report for Phi3:")
print(class_report_phi3)
print("\nClassification Report for Gemma2:")
print(class_report_gemma2)
print("\nClassification Report for Llama:")
print(class_report_llama)



Classification Report for Phi3:
              precision    recall  f1-score   support

    positive       0.93      0.36      0.52        78
    negative       0.83      0.49      0.61        39
       mixed       0.52      0.93      0.67        83

    accuracy                           0.62       200
   macro avg       0.76      0.59      0.60       200
weighted avg       0.74      0.62      0.60       200


Classification Report for Gemma2:
              precision    recall  f1-score   support

    positive       0.96      0.29      0.45        78
    negative       0.84      0.41      0.55        39
       mixed       0.50      0.95      0.66        83

    accuracy                           0.59       200
   macro avg       0.77      0.55      0.55       200
weighted avg       0.75      0.59      0.56       200


Classification Report for Llama:
              precision    recall  f1-score   support

    positive       0.88      0.47      0.62        78
    negative       0.85    